# **Computer Vision Mini Project - Tomato Ripeness Detection**

Instruction: If you want to train the model again from the start, please import this ipynb file into your Google Colab and run it there.

In this ZIP, I have imported a fully trained model from this ipynb, this zip function is only to run its image, video, and realtime webcam test.


---



**Dataset information:**
https://www.kaggle.com/datasets/nexuswho/tomatod

Class: unripe, semi-ripe, fully-ripe

Total class: 277

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kenkenma","key":"e6dad43d14005c638a198c638e7f5b9e"}'}

In [2]:
!pip install ultralytics

from ultralytics import YOLO
import os
import shutil
import random
from glob import glob

!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [40]:
!kaggle datasets download -d nexuswho/tomatod -p dataset/ --unzip

Dataset URL: https://www.kaggle.com/datasets/nexuswho/tomatod
License(s): CC-BY-NC-SA-4.0
 89% 145M/164M [00:01<00:00, 92.0MB/s]
100% 164M/164M [00:01<00:00, 105MB/s] 


In [41]:
!mv "/content/dataset/images/val" "/content/dataset/images/test"
!mv "/content/dataset/labels/val" "/content/dataset/labels/test"

In [42]:
print("Images numbers: train and test")
!ls /content/dataset/images/train| wc -l
!ls /content/dataset/images/test | wc -l

print("Labels numbers: train and test")
!ls /content/dataset/labels/train| wc -l
!ls /content/dataset/labels/test | wc -l

Images numbers: train and test
222
55
Labels numbers: train and test
222
55


In [43]:
original_dataset = "/content/dataset"
new_dataset = "/content/dataset_new"

train_images = os.path.join(original_dataset, "images/train")
train_labels = os.path.join(original_dataset, "labels/train")
test_images = os.path.join(original_dataset, "images/test")
test_labels = os.path.join(original_dataset, "labels/test")

new_train_images = os.path.join(new_dataset, "images/train")
new_train_labels = os.path.join(new_dataset, "labels/train")
new_val_images = os.path.join(new_dataset, "images/val")
new_val_labels = os.path.join(new_dataset, "labels/val")
new_test_images = os.path.join(new_dataset, "images/test")
new_test_labels = os.path.join(new_dataset, "labels/test")

for path in [new_train_images, new_train_labels, new_val_images, new_val_labels, new_test_images, new_test_labels]:
    os.makedirs(path, exist_ok=True)

train_files = sorted(os.listdir(train_images))
train_labels_files = sorted(os.listdir(train_labels))

class_distribution = {"unripe": [], "semi-ripe": [], "fully-ripe": []}

for img in train_files:
    label_path = os.path.join(train_labels, img.replace(".jpg", ".txt"))
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            class_id = int(f.readline().split()[0])
        class_name = ["unripe", "semi-ripe", "fully-ripe"][class_id]
        class_distribution[class_name].append(img)

selected_val = []
for class_name in class_distribution:
    selected_val.extend(random.sample(class_distribution[class_name], min(7, len(class_distribution[class_name]))))

for img in selected_val:
    shutil.move(os.path.join(train_images, img), os.path.join(new_val_images, img))
    shutil.move(os.path.join(train_labels, img.replace(".jpg", ".txt")), os.path.join(new_val_labels, img.replace(".jpg", ".txt")))

remaining_train_images = os.listdir(train_images)
for img in remaining_train_images:
    shutil.move(os.path.join(train_images, img), os.path.join(new_train_images, img))
    shutil.move(os.path.join(train_labels, img.replace(".jpg", ".txt")), os.path.join(new_train_labels, img.replace(".jpg", ".txt")))

for img in os.listdir(test_images):
    shutil.copy(os.path.join(test_images, img), os.path.join(new_test_images, img))
for lbl in os.listdir(test_labels):
    shutil.copy(os.path.join(test_labels, lbl), os.path.join(new_test_labels, lbl))

shutil.rmtree(original_dataset)

print("Dataset successfully split and moved to dataset_new!")

Dataset successfully split and moved to dataset_new!


In [44]:
print("Images numbers: train, val, and test")
!ls /content/dataset_new/images/train| wc -l
!ls /content/dataset_new/images/val | wc -l
!ls /content/dataset_new/images/test | wc -l

print("Labels numbers: train, val, and test")
!ls /content/dataset_new/labels/train| wc -l
!ls /content/dataset_new/labels/val | wc -l
!ls /content/dataset_new/labels/test | wc -l

Images numbers: train, val, and test
201
21
55
Labels numbers: train, val, and test
201
21
55


In [45]:
data_yaml_content = """
train: "/content/dataset_new/images/train"
val: "/content/dataset_new/images/val"
test: "/content/dataset_new/images/test"
nc: 3
names: ['unripe', 'semi-ripe', 'fully-ripe']
"""

yaml_path = "/content/dataset_new/data.yaml"

with open(yaml_path, "w") as f:
    f.write(data_yaml_content)

print(f"data.yaml created successfully at {yaml_path}!")

data.yaml created successfully at /content/dataset_new/data.yaml!


In [46]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/dataset_new/data.yaml epochs=120 imgsz=640 optimizer=Adam lr0=0.001 batch=16

100% 49.7M/49.7M [00:00<00:00, 126MB/s]
Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/dataset_new/data.yaml, epochs=120, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,

In [47]:
!yolo task=detect mode=val model=runs/detect/train/weights/best.pt data=/content/dataset_new/data.yaml

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/dataset_new/labels/val.cache... 21 images, 0 backgrounds, 0 corrupt: 100% 21/21 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:01<00:00,  1.37it/s]
                   all         21        151      0.777      0.827      0.837       0.57
                unripe         21        107      0.802      0.935      0.948      0.594
             semi-ripe         16         22      0.829      0.727      0.793      0.609
            fully-ripe         16         22        0.7      0.818      0.771      0.507
Speed: 5.5ms preprocess, 26.2ms inference, 0.0ms loss, 11.2ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [48]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt source=/content/dataset_new/images/test/

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs

image 1/55 /content/dataset_new/images/test/stereo20190405_132743_p0_snap_514.jpg: 640x640 14 unripes, 1 semi-ripe, 1 fully-ripe, 37.0ms
image 2/55 /content/dataset_new/images/test/stereo20190405_133623_p0_snap_043.jpg: 640x640 9 unripes, 1 semi-ripe, 1 fully-ripe, 37.1ms
image 3/55 /content/dataset_new/images/test/stereo20190405_133623_p0_snap_089.jpg: 640x640 5 unripes, 1 semi-ripe, 37.0ms
image 4/55 /content/dataset_new/images/test/stereo20190405_133623_p0_snap_092.jpg: 640x640 4 unripes, 35.9ms
image 5/55 /content/dataset_new/images/test/stereo20190405_133623_p0_snap_180.jpg: 640x640 4 unripes, 2 semi-ripes, 3 fully-ripes, 34.7ms
image 6/55 /content/dataset_new/images/test/stereo20190405_133623_p1_snap_024.jpg: 640x640 3 unripes, 34.7ms
image 7/55 /content/dataset_new/images/test/stereo20190405_133623_p1_snap_027.jpg: 

In [51]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt source=/content/tomatoes_test.mp4

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs

video 1/1 (frame 1/195) /content/tomatoes_test.mp4: 640x384 5 unripes, 2 fully-ripes, 90.7ms
video 1/1 (frame 2/195) /content/tomatoes_test.mp4: 640x384 2 unripes, 3 fully-ripes, 25.3ms
video 1/1 (frame 3/195) /content/tomatoes_test.mp4: 640x384 4 unripes, 1 fully-ripe, 25.2ms
video 1/1 (frame 4/195) /content/tomatoes_test.mp4: 640x384 4 unripes, 1 fully-ripe, 25.2ms
video 1/1 (frame 5/195) /content/tomatoes_test.mp4: 640x384 2 unripes, 2 fully-ripes, 25.1ms
video 1/1 (frame 6/195) /content/tomatoes_test.mp4: 640x384 (no detections), 25.1ms
video 1/1 (frame 7/195) /content/tomatoes_test.mp4: 640x384 2 unripes, 25.1ms
video 1/1 (frame 8/195) /content/tomatoes_test.mp4: 640x384 1 unripe, 25.7ms
video 1/1 (frame 9/195) /content/tomatoes_test.mp4: 640x384 3 unripes, 1 fully-ripe, 15.6ms
video 1/1 (frame 10/195) /content/tomato

In [52]:
from google.colab import files

folder_to_zip = "/content/runs"
output_zip = "/content/runs.zip"

shutil.make_archive(output_zip.replace(".zip", ""), 'zip', folder_to_zip)

files.download(output_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Instruction:

After you got this mini project folder, please unzip it and open the 'models.py' in your Visual Studio Code, run it to unzip the runs.zip files and access the result of train, val, and prediction in images and videos.